# TensorFlow Notes

## Installation

Just follow the official guide. <https://www.tensorflow.org/install/>

## Tensors

A 'tensor' in TensorFlow is just <b>an N-dimensional array</b>. A tensor's <b>rank</b> is its number of dimensions. Tensor is the central unit of data in TensorFlow.

Some examples:

In [1]:
tensor = 3                                                 # rank=0, shape=()
tensor = [1, 2, 3]                                         # rank=1, shape=(3)
tensor = [[1, 2, 3], [4, 5, 6]]                            # rank=2, shape=(2, 3)
tensor = [[[1, 2, 3], [3, 2, 1]], [[4, 5, 6], [6, 5, 4]]]  # rank=3, shape=(2, 2, 3)
tensor = [[[1, 2, 3]], [[4, 5, 6]]]                        # rank=3, shape=(2, 1, 3)

## TensorFlow core tutorial

### Importing TensorFlow in Python

In [2]:
import tensorflow as tf

This is the convention when importing TensorFlow, and it gives Python access to all of tensorflow's classes, methods, and symbols.

### The Computational Graph

The TensorFlow Core programs consist of two discrete sections:

1. Building the computational graph
2. Running the computational graph

A <b>computational graph</b> is a series of TensorFlow operations arranged into a graph of <b>nodes</b>. Each node takes zero or more tensors as inputs and produces a tensor as an output.

#### Constant node

We cannot change the value of a node when it has type <b>constant</b>.

In [3]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # tf.float32 by default
print node1, node2

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


We might expect the output to be (3.0, 4.0), which is not the case in the actual output. These values can be revealed when we run the nodes inside a TensorFlow <b>session</b>. A session encapsulates the control and state of the Tensorflow runtime.

In [4]:
sess = tf.Session()
print sess.run([node1, node2])

[3.0, 4.0]


#### Nodes with operations

In [5]:
node3 = tf.add(node1, node2) # node for addition
print "node3:", node3
print "sess.run(node3):", sess.run(node3)

node3: Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3): 7.0


#### Placeholders

A <b>placeholder</b> is a data holder to which you don't have to give a certain value immediately. You can provide a value later.

In [6]:
a = tf.placeholder(tf.float32) # don't need to initialise
b = tf.placeholder(tf.float32)
adder_node = a + b # shortcut for tf.add(a, b)

print sess.run(adder_node, {a:3, b:4.5}) # provide values when in session
print sess.run(adder_node, {a:[1, 3], b:[2, 4]})

7.5
[ 3.  7.]


The operation can become more complex:

In [7]:
add_and_triple = adder_node*3
print sess.run(add_and_triple, {a:3, b:4.5})

22.5


#### Variables

In contrast to constants, <b>variables</b> can be adjusted by the program. Variables allow us to add trainable parameters to a graph. They are constructed with a type and initial value:

In [8]:
W = tf.Variable([0.3], dtype=tf.float32)
b = tf.Variable([-0.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W*x + b

However, it is important to note that the global values in TensorFlow are uninitialised until we first call `sess.run`. Therefore, to initialise all the variables in a TensorFlow program, we must explicitly call a special operation as follows:

In [9]:
init = tf.global_variables_initializer()
sess.run(init)

Since x is a placeholder, we can evaluate `linear model` for several values of `x` simultaneously as follows:

In [10]:
print sess.run(linear_model, {x:[1, 2, 3, 4]})

[ 0.          0.30000001  0.60000002  0.90000004]


#### Loss function

A <b>loss function</b> measures how far apart the current model is from the provided data. For example, it can be the sum of squared error:

In [11]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print sess.run(loss, {x:[1, 2, 3, 4], y:[0, -1, -2, -3]})

23.66


Actually, we can change `W` and `b` to provide a better fit:

In [12]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print sess.run(loss, {x:[1, 2, 3, 4], y:[0, -1, -2, -3]})

0.0


We guessed the 'perfect' values of `W` and `b`, but the whole point of machine learning is to find the correct model parameters automatically. We will show how to accomplish this in the next section.

## tf.train API

TensorFlow provides <b>optimizers</b> that slowly change each variable in order to minimize the loss function. The simplest optimizer is <b>gradient descent</b>. It modifies each variable according to the magnitude of the derivative of loss with respect to that variable.

In [13]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [14]:
sess.run(init) # reset values to incorrect defaults
for i in range(1000):
    sess.run(train, {x:[1, 2, 3, 4], y:[0, -1, -2, -3]})

print sess.run([W, b])

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


### Complete program

In [16]:
import numpy as np
import tensorflow as tf

# Model parameters
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)
# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
# training data
x_train = [1,2,3,4]
y_train = [0,-1,-2,-3]
# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
    sess.run(train, {x:x_train, y:y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x:x_train, y:y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11
